 <span style='background:gray;font-size:40px;'>  Setup  </span>

 <span style='background:gray;font-size:40px;'>  Importing Libraries  </span>

In [2]:

from google.cloud import storage

import os
import random
import json
import gzip
import csv
import itertools

import numpy as np
import pandas as pd

from urllib.request import urlopen
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
#from amazon.api import AMAZONAPI

 <span style='background:gray;font-size:40px;'>  Jupyter Options  </span>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_colwidth', 100)

In [ ]:
# Already running in my instance since I checkboxed Anaconda
#%pip install more-itertools

***

# Import from Bucket - (Not needed since i uploaded directly onto persistent SSD)

In [ ]:
bucket_name = "finalproject-storage"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# # When you have your files in a subfolder of the bucket.
# my_prefix = "/" # the name of the subfolder
# blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

# for blob in blobs:
#     if(blob.name != my_prefix): # ignoring the subfolder itself 
#         file_name = blob.name.replace(my_prefix, "")
#         blob.download_to_filename(file_name) # download the file to the machine
#         df = pd.read_csv(file_name) # load the data
#         print(df)

# When you have your files on the first level of your bucket

blobs = bucket.list_blobs()


for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name) # download the file to the machine
    # df = pd.read_csv(file_name) # load the data
    # print(df)
    
# elements = bucket.list_blobs()
# files=[a.name for a in elements]
# files
# path=f"gs://finalproject-storage/notebooks/jupyter/xaa"

In [ ]:
# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('alouis9')
blob = bucket.blob('Electronics_5.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
#json_data = blob.download_as_string().decode('utf-8')


In [ ]:
bucket.list_blobs()
blob
bucket

In [ ]:
for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name)

In [ ]:
data = pd.read_csv("gs://alouis9/Electronics_5.json")

***

<span style='background:gray;font-size:40px;'>  View Local Persistent SSD Tmp Folder where datafiles are stored  </span>

In [ ]:
import os
[s for s in os.listdir('/home/derek') if (s.endswith('.csv') | s.endswith('.gz'))]


#os.listdir



In [11]:
theFiles

['meta_Electronics.csv',
 'edgelist.csv',
 'All_Amazon_Meta.json.gz',
 'Musical_Instruments_5.json.gz',
 'meta_Electronicscleaned_.csv',
 'Electronics.json.gz',
 'Electronics_5.json.gz',
 'Musical_Instruments.csv',
 'nodeslist.csv',
 'Sports_and_Outdoors.csv',
 'meta_Electronics.json.gz',
 'meta_Sports_and_Outdoors.json.gz',
 'title_alsobuy_asin_master.csv',
 'Electronics.csv']

In [13]:
path='/home/derek/'
theFiles = [s for s in os.listdir('/home/derek') if (s.endswith('.csv') | s.endswith('.gz'))]  #list(os.listdir('/'))

theDict = dict()
for something in theFiles: #Calculate size for all files here. 
    theStats = os.stat(path+something)
    theDict[something] = theStats

for item in theDict:
    print("The File: {:30s}                  The Size: {:d} Bytes".format(item,theDict[item].st_size))

The File: meta_Electronics.csv                            The Size: 121201747 Bytes
The File: edgelist.csv                                    The Size: 4765955 Bytes
The File: All_Amazon_Meta.json.gz                         The Size: 12327618560 Bytes
The File: Musical_Instruments_5.json.gz                   The Size: 39356138 Bytes
The File: meta_Electronicscleaned_.csv                    The Size: 64040023 Bytes
The File: Electronics.json.gz                             The Size: 3322874357 Bytes
The File: Electronics_5.json.gz                           The Size: 1251876861 Bytes
The File: Musical_Instruments.csv                         The Size: 63130088 Bytes
The File: nodeslist.csv                                   The Size: 196103 Bytes
The File: Sports_and_Outdoors.csv                         The Size: 541795338 Bytes
The File: meta_Electronics.json.gz                        The Size: 1198601797 Bytes
The File: meta_Sports_and_Outdoors.json.gz                  The Size: 523414200

# Nodes List w Image Link

In [14]:
    path='/home/derek/meta_Sports_and_Outdoors.json.gz'
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)
    def getDF(whatev):
        i = 0
        df = {}
        for d in parse(whatev):
            df[i] = d
            i += 1
        return pd.DataFrame.from_dict(df, orient='index')

    dfnodes = getDF(path)
    dfnodes = dfnodes[['asin', 'title', 'also_buy','date', 'imageURLHighRes']].explode('imageURLHighRes')
    dfnodes = dfnodes[dfnodes.also_buy.str.len() > 0]   
    dfnodes = dfnodes.groupby('asin').first().reset_index()
    dfnodes['also_buys'] = dfnodes['also_buy'].agg(lambda x: ','.join(map(str, x)))
    dfnodes = dfnodes.sort_values(by='date', ascending=False).drop_duplicates(subset=['title','also_buys'], keep='first')
    dfnodes.to_csv(path.split('.')[0] + 'cleaned_' + '.csv', index=False)

In [15]:
dfnodes.head()
dfnodes[dfnodes.duplicated('asin')]
print("{}{:,}{}".format("There are ",dfnodes.asin.count()," unique asins w metadata \n"))


,asin,title,also_buy,date,imageURLHighRes,also_buys
261572,B01H3MRD48,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,"[B07J9QSVVX, B01LNJDK5S, B017DC1X6C, B01D2W7NFW, B017CQLJ04, B01H1LCCG0, B01LA9B65W, B008031QKW]",yunanwa,https://images-na.ssl-images-amazon.com/images/I/41Swby3LkzL.jpg,"B07J9QSVVX,B01LNJDK5S,B017DC1X6C,B01D2W7NFW,B017CQLJ04,B01H1LCCG0,B01LA9B65W,B008031QKW"
96769,B00BNFNU0S,Silicone Slap-On Watch - Quartz Authentic Slap Watch - Blue,"[B078YS9SKB, B07JMQM7YB, B01BYAZCXO, B01BYAJUYQ, B003EIK136, B005SHDGI8, B004UG4I3K, B005SHD0NO,...",vibeWEAR,https://images-na.ssl-images-amazon.com/images/I/41q7eG0ju1L.jpg,"B078YS9SKB,B07JMQM7YB,B01BYAZCXO,B01BYAJUYQ,B003EIK136,B005SHDGI8,B004UG4I3K,B005SHD0NO,B01BYATL..."
204227,B0147EN9SO,His Doe Ring &quot;for Women&quot; - Tungsten - Couples Ring Set - Hunting - Ladies Ring,"[B0147E5NRE, B014LPWOAS, B07DKXG1HX]",southern designs,None,"B0147E5NRE,B014LPWOAS,B07DKXG1HX"
204225,B0147E5NVA,southern designs Buck and Doe Ring Band Ring in His and Her Sizes - Tungsten,"[B0147EN9U2, B014LPWOAS]",southern designs,None,"B0147EN9U2,B014LPWOAS"
244303,B01DJ61XWK,SONGDU Date Black Multifunction Chronograph Mens Watches Stopwatch Rose Gold With Leather Strap ...,"[B0719D5GXB, B078N4DPT7, B0777HBD4D, B01F290W92, B07MJ1T5ZR, B071FTG25L, B07MDT2Q46, B07FPJDJ29,...",songdu binger,https://images-na.ssl-images-amazon.com/images/I/415ITq63ysL.jpg,"B0719D5GXB,B078N4DPT7,B0777HBD4D,B01F290W92,B07MJ1T5ZR,B071FTG25L,B07MDT2Q46,B07FPJDJ29,B0767DRW..."


,asin,title,also_buy,date,imageURLHighRes,also_buys


There are 249,165 unique asins w metadata 




<span style='background:red;font-size:40px;'>  Import Ratings Only CSV Data  </span>
***

In [16]:
path = '/home/derek/Sports_and_Outdoors.csv'
header_list = ["item", "user", "rating", "timestamp"]
types_dict = {'item': 'string', 'user': 'string', 'rating': float, 'timestamp':'string'}
df_ratings = pd.read_csv(path, names=header_list, dtype=types_dict)
print("{}{:,}{}".format("There are ",len(df_ratings)," reviews in this dataset \n"))
df_ratings.info()
df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])[["item", "user", "rating", "timestamp", "title"]]
df_ratings.head()

There are 12,980,837 reviews in this dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12980837 entries, 0 to 12980836
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   item       string 
 1   user       string 
 2   rating     float64
 3   timestamp  string 
dtypes: float64(1), string(3)
memory usage: 396.1 MB


,item,user,rating,timestamp,title
0,0000031895,A23K73OVXJ04EG,5.0,1391212800,Girls Ballet Tutu Neon Blue
1,0000031895,A2681T699HV6H1,4.0,1384905600,Girls Ballet Tutu Neon Blue
2,0000031895,A374PA18DCGS5Y,1.0,1477008000,Girls Ballet Tutu Neon Blue
3,0000031895,A14PVW2N5YBWSA,5.0,1476748800,Girls Ballet Tutu Neon Blue
4,0000031895,A2KWBC44QI2567,1.0,1476662400,Girls Ballet Tutu Neon Blue



<span style='background:green;font-size:40px;'> Metadata Exploration  </span>
***



<span style='background:green;font-size:40px;'> Rating Frequency Count </span>


In [17]:
#df_ratings.style.format('{:,}')
ratingsFreq = df_ratings.groupby('rating').count().sort_values(['rating'],ascending=False)
ratingsFreq.style.format('{:,}')
print("{:,}{}{:,}{}".format(ratingsFreq.item.iloc[0]," out of ", ratingsFreq.item.sum(), " total reviews were rated 5 stars \n"))

,item,user,timestamp,title
rating,,,,
5.0,"8,351,612","8,351,612","8,351,612","6,247,960"
4.0,"2,047,306","2,047,306","2,047,306","1,469,402"
3.0,"968,231","968,231","968,231","676,574"
2.0,"617,198","617,198","617,198","425,115"
1.0,"996,490","996,490","996,490","666,088"


8,351,612 out of 12,980,837 total reviews were rated 5 stars 



In [18]:
ratingspct = df_ratings.rating.value_counts(normalize=True)
print("{}{:,.1%}{}".format("",ratingspct.iloc[0]," of the reviews were rated 5 stars \n"))
ratingspct.to_frame().style.format('{:,.1%}')

64.3% of the reviews were rated 5 stars 



,rating
5.0,64.3%
4.0,15.8%
1.0,7.7%
3.0,7.5%
2.0,4.8%



<span style='background:green;font-size:40px;'> Average Rating Per Product Sorted by # of reviews </span>

In [54]:
pd.set_option('display.max_rows', 5)

In [57]:

df_avgrev = df_ratings.groupby(['title','item']).agg(dict(rating='mean', user='count')).sort_values(by=['rating','user'], ascending=False)
df_avgrev[df_avgrev.rating==5.0].head(5)

,,rating,user
title,item,,
"KAIMI Training Gear : Jump Ropes, Workout Mask, Boxing Ball | Reflex Speed Headband Exercise Fitness Cardio MMA Skipping Jumprope Reaction Altitude Elevation Gym Running Fight Breathing",B00ERZ562Q,5.0,129
"BRAVO COMPANY BCM Keymod Aluminum Picatinny Rail Section, Black, 3&quot;",B00ITWLN3E,5.0,48
SHIMANO Dura-Ace Grease,B00177DBBA,5.0,48
RaceFace Atlas Mountain Bike Handlebar,B008ZU19UW,5.0,46
Yogi Junior Kids Yoga Mat - PVC Free - Double Layered TPE Foam,B01E4HU5YK,5.0,40


In [46]:
df_ratings.groupby(['title','item'])['rating'].mean().sort_values(ascending=False)

title                                                                                   item      
Maxxis Torch Black Wire/120 DC/SS Tires                                                 B00PHVNDZG    5.0
Discraft Z Drone Mid Range Golf Disc                                                    B00EDK75V0    5.0
Discraft heat                                                                           B01FHYTBQ2    5.0
Raintropical Elegant Inspired Retro Vintage One Piece Pin Up Monokinis Swimsuit(FBA)    B01H27YXEM    5.0
Discraft Zone ESP Golf Disc                                                             B0035N62JW    5.0
                                                                                                     ... 
Personalized Custom Football Aluminum Silver Finish 20 Ounce Water Bottle Customizable  B015CTR3HG    1.0
Eskimo Engine Oil 4-Cycle Engine Oil                                                    B018STG968    1.0
Perri's Field Boot Laces, Black, 35-Inch             

<span style='background:green;font-size:40px;'> Top 25 Most Reviewed Products </span>

In [20]:
#num_reviews = df_ratings.groupby('item').count()
#num_reviews.sort_values(['rating'],ascending=False)
df_ratings.groupby(['title','item']).size().sort_values(ascending=False).head(25)#.agg('count')
#top100reviewers = rev_prolific.head(100).reset_index().copy()
#top100reviewers

title                                                                                                                                                                               item      
Converse Chuck Taylor All Star Low Top                                                                                                                                              B00V6JAEDC    10226
Thermos Stainless King 40 Ounce Beverage Bottle, Midnight Blue                                                                                                                      B0017IHRNM     9472
Intex Comfort Plush Elevated Dura-Beam Airbed with Built-in Electric Pump                                                                                                           B00G7H793G     8801
3M WorkTunes Hearing Protector with AM/FM Radio                                                                                                                                     B0013092CS     8592
BalanceFr

<span style='background:green;font-size:40px;'> Top 10 Product with the most reviews per Rating </span>

In [56]:

#df_ratings.groupby(['rating', 'item']).size().sort_values(ascending=False).nlargest(10)
ratagg = df_ratings.groupby(['title','rating', 'item']).agg({'user':'nunique'}).reset_index()
ratsort = ratagg.groupby(['rating']).apply(lambda x: x.sort_values(['user'],ascending = False)).reset_index(drop = True)
ratsort.groupby(['rating']).head(100).sort_values(['rating', 'user'],ascending=False)


,title,rating,item,user
462741,Converse Chuck Taylor All Star Low Top,5.0,B00V6JAEDC,6884
462742,"Hydro Flask Insulated Wide Mouth Stainless Steel Water Bottle, 32-Ounce",5.0,B004X55L9I,6584
462743,"Vont 4 Pack LED Camping Lantern, Survival Kit for Hurricane, Emergency, Storm, Outages, Outdoor ...",5.0,B00NPLSZF8,6406
462744,"Vont 2 Pack LED Camping Lantern, Portable, Great Addition to: Survival Kits Hurricane, Emergency...",5.0,B018L2WM86,6194
462745,BalanceFrom GoYoga All-Purpose 1/2-Inch Extra Thick High Density Anti-Tear Exercise Yoga Mat wit...,5.0,B00FO9ZRYQ,5790
...,...,...,...,...
95,"Generation 4 Sure Plug for Glock 17, 19, 22, 23, 31, 32, 34, and 35",1.0,B0062CB360,236
96,CamelBak eddy Water Bottle (Discontinued Styles),1.0,B006ZT936W,232
97,CAP Barbell Flat/Incline/Decline Bench,1.0,B001IKHUNQ,232
98,"Wahoo TICKR Heart Rate Monitor, Bluetooth / ANT+",1.0,B00INQVYZ8,230


<span style='background:green;font-size:40px;'> Top 25 Most Prolific Reviewers </span>

In [22]:
# rev_prolific = df_ratings.groupby('user').count().sort_values(['rating'],ascending=False)
# top100reviewers = rev_prolific.head(100).reset_index().copy()
# top100reviewers

df_ratings.groupby('user').size().sort_values(ascending=False).head(25)

user
A3OXHLG6DIBRW8    433
AN81JUYW2SL24     373
AVU1ILDDYW301     347
A2O489VAPVIH35    338
A8VI7KMUHI7ZH     310
AV0B6NGGOAISO     302
AGGAQLXPVGM5D     288
A33YZNZIRA3H97    260
A2MR7JGGFD394J    253
ACSA4CBJMEE4O     252
A2AF0ISXVVB6SP    251
ASWTGWOE7YB9T     248
A29HMA1ATAK4MB    243
A3L1VJMHFWONCB    242
A2503LT8PZIHAD    229
A259MPG6SOL02P    215
A2L5D2ZY6GVBFA    215
A2XX2A4OJCDNLZ    215
A1RPTVW5VEOSI     211
A1JZFGZEZVWQPY    207
A1H6SBO7R007I8    207
A26KNV0HTA4U5D    202
AXUJFOFQZNTN      193
A3NHUQ33CFH3VM    182
A3C1QYGEET3BVY    175
dtype: int64

# Find The Product with most reviewer agreement for Each Rating

 <span style='background:red;font-size:40px;'> Load Category Product-Level Meta Data and create DF </span>

In [59]:
path = '/home/derek/meta_Sports_and_Outdoors.json.gz'
metadata = []
with gzip.open(path) as f:
    for l in f:
        metadata.append(json.loads(l.strip()))

In [60]:
# convert list into pandas dataframe
df_metadata = pd.DataFrame.from_dict(metadata)
df_metadata['also_buys'] = df_metadata['also_buy'].agg(lambda x: ','.join(map(str, x)))
df_metadata = df_metadata.sort_values(by='date', ascending=False).drop_duplicates(subset=['title','also_buys'], keep='first')
print(len(df_metadata))
df_metadata.columns
len(df_metadata[df_metadata.isna().any(axis=1)]) # Rows with NANS
df_metadata.info()


925771


Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details',
       'also_buys'],
      dtype='object')

137

<class 'pandas.core.frame.DataFrame'>
Int64Index: 925771 entries, 956477 to 962299
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         925771 non-null  object
 1   tech1            925771 non-null  object
 2   description      925771 non-null  object
 3   fit              925771 non-null  object
 4   title            925771 non-null  object
 5   also_buy         925771 non-null  object
 6   tech2            925771 non-null  object
 7   brand            925771 non-null  object
 8   feature          925771 non-null  object
 9   rank             925771 non-null  object
 10  also_view        925771 non-null  object
 11  main_cat         925771 non-null  object
 12  similar_item     925771 non-null  object
 13  date             925771 non-null  object
 14  price            925771 non-null  object
 15  asin             925771 non-null  object
 16  imageURL         925771 non-null  object
 17  image

In [61]:
type(metadata)
len(metadata)
type(df_metadata)
len(df_metadata)

list

962300

pandas.core.frame.DataFrame

925771

In [62]:
print("{}{:,}{}".format("There are ",len(metadata)," products w metadata \n"))

# first row of the list
print(metadata[0])

There are 962,300 products w metadata 

{'category': ['Sports & Outdoors', 'Sports & Fitness', 'Other Sports', 'Dance', 'Clothing'], 'tech1': '', 'description': ['3 layers of super-soft polyester tulle can be cut to length. Satin covered stretch elastic waist gives a one size fits all most.'], 'fit': '', 'title': 'Adult Tutu Assorted Colors (Turquoise)', 'also_buy': [], 'tech2': '', 'brand': 'BubuBibi', 'feature': ['3 Layers - 100% Polyester Tulle, Hand Wash Lay FLat to Dry', '100% Polyester', 'FIT: most Kids. Teens . Adults *Please Check Measurements for Proper Fit', 'WAIST: Satin Covered Elastic 18" relaxed to 36" fully stretched', 'LENGTH: 16" to 17" *can easily be cut to desired length without worry of fraying or unraveling', 'You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.', '<a id="creturns-policy-url-text-pc" class="a-link-normal" href="/b/ref=frt_dp_sm?ie=UTF8&node=8622296011">\n         

 <span style='background:red;font-size:40px;'> Load MASTER Product-Level Meta Data (all categories) and create DF </span>

In [63]:
path = '/home/derek/title_alsobuy_asin_master.csv'
col_list = ["title", "asin"]
types_dict = {'title': 'string', 'alsobuy': 'string', 'asin': 'string'}
df_masterMETA = pd.read_csv(path, dtype=types_dict, usecols=col_list, skipinitialspace = True)

In [64]:
print("{}{:,}{}".format("There are ",len(df_masterMETA)," items in this dataset \n"))
df_masterMETA.info()
#df_masterMETA = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])#[["item", "user", "rating", "timestamp", "title"]]
df_masterMETA.head()

There are 15,023,059 items in this dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15023059 entries, 0 to 15023058
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   title   string
 1   asin    string
dtypes: string(2)
memory usage: 229.2 MB


,title,asin
0,"Women Blouse, Ninasill Hooded Sweatshirt Coat Winter Warm Wool Zipper Pockets Cotton Coat Outwear",6305121869
1,Coolred-Women Retro Split Sexy Skinny Premium Qipao Maxi Dress Cheongsam,6318708057
2,Gaok Men's Retro Cotton Multi-Pocket Camo Cargo Shorts,6342506256
3,QIBOE Men's Baggy Jeans Denim Sweatpants Loose Pants,6342509379
4,Crazy Explosion-proof Lens Polarized Sunglasses Cycling Glasses Lenses,6342522081


<span style='background:red;font-size:40px;'>Build Subset df to eliminate asins with no alsobuys</span>

In [65]:
df_alsobuys = df_metadata[df_metadata.also_buy.str.len() > 0]
df_alsobuys[df_alsobuys.also_buy.isna()]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


In [66]:
df_metadata = df_alsobuys

 <span style='background:green;font-size:40px;'>Duplicate ASINS in Metadata Table</span>

In [67]:
#df_metadata[df_metadata.duplicated('asin')].head()
len(df_metadata[df_metadata.duplicated('asin')])

0

<span style='background:green;font-size:40px;'> List of Duplicate asins in metadata</span>

In [68]:


asin_dupes = df_metadata[df_metadata.duplicated('asin')].sort_values(by=['asin']).asin.tolist()

len(asin_dupes)

asin_dupes[0:10]
print("{}{:,}{}".format("There are ",len(asin_dupes)," asins with more than one row \n"))


0

[]

There are 0 asins with more than one row 




<span style='background:green;font-size:40px;'>View Actual Duplicate Asins Side by Side in Metadata Table</span>

In [69]:


df_asindupes = pd.concat(g for _, g in df_metadata.groupby('asin') if len(g) > 1)
df_asindupes.shape
print("{}{:,}{}{:,}{}".format("Since there are ",len(asin_dupes)," asins with more than one row, there are ", len(df_asindupes)," rows in total \n"))
df_asindupes.head(10)


ValueError: No objects to concatenate

# These are the rows with NAN in Details Column

In [70]:
#df_metadata[df_metadata.isna().any(axis=1)]

In [71]:
df_metadata[df_metadata.asin=='B01NAJ3KQB']

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


 <span style='background:red;font-size:40px;'> ***FILTER METADATA DF***</span>

In [72]:
df_metadata[df_metadata.asin=='B00JHKSMG8']

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


***

 <span style='background:orange;font-size:40px;'>Co-purchases Search</span>

In [73]:
no_copurch = len(df_metadata[(df_metadata.also_buy.str.len() == 0) & (df_metadata.also_view.str.len() == 0)])
both_copurch = len(df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)])

print("{}{:,}{}".format("There are ",len(df_metadata)," asins in total in the metadata table \n"))

print("{}{:,}{}".format("There are ",no_copurch," asins with no also_buys or also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() == 0])," asins with no also_buys \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() == 0])," asins with no also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() > 0])," asins with at least one also_views co-purchase \n"))

print("{}{:,}{}".format("There are ",both_copurch," asins with at least one of both co-purchase types \n"))

df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)].head()
df_graph = df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)]


There are 249,165 asins in total in the metadata table 

There are 0 asins with no also_buys or also_views 

There are 0 asins with no also_buys 

There are 58,217 asins with no also_views 

There are 249,165 asins with at least one also_buy co-purchase 

There are 190,948 asins with at least one also_views co-purchase 

There are 190,948 asins with at least one of both co-purchase types 



,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys
956477,"[Sports & Outdoors, Outdoor Recreation, Outdoor Clothing, Men, Accessories, Sport Watches, Packa...",,"[Features: <br/>Shows time digital numbers <br/>Display time in numbers indicate the hour, minut...",,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,"[B07J9QSVVX, B01LNJDK5S, B017DC1X6C, B01D2W7NFW, B017CQLJ04, B01H1LCCG0, B01LA9B65W, B008031QKW]",,yunanwa,"[Package Included:10 Pack Led Watches 2016 Update Version, Additional RED LED function to show t...","680,789 in Clothing, Shoes & Jewelry (","[B01LNJDK5S, B06XD7FZWV, B01D2W7NFW, B07J9QSVVX, B01EN7RUHG, B01CJLLM6S, B017DC1X6C]","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,yunanwa,$19.99,B01H3MRD48,"[https://images-na.ssl-images-amazon.com/images/I/41Swby3LkzL._SR38,50_.jpg, https://images-na.s...","[https://images-na.ssl-images-amazon.com/images/I/41Swby3LkzL.jpg, https://images-na.ssl-images-...",{},"B07J9QSVVX,B01LNJDK5S,B017DC1X6C,B01D2W7NFW,B017CQLJ04,B01H1LCCG0,B01LA9B65W,B008031QKW"
420887,"[Sports & Outdoors, Fan Shop, Jewelry & Watches]",,"[As fun as it is functional, this slap watch features a stiff band that ""slaps"" around your wris...",,Silicone Slap-On Watch - Quartz Authentic Slap Watch - Blue,"[B078YS9SKB, B07JMQM7YB, B01BYAZCXO, B01BYAJUYQ, B003EIK136, B005SHDGI8, B004UG4I3K, B005SHD0NO,...",,vibeWEAR,"[100% Silicone Band - Fits 99% of Wrists, Quartz Time-Piece for 100% Accuracy, Lightweight and C...","529,043 in Clothing, Shoes & Jewelry (","[B078YS9SKB, B07K68NDPY, B07JMQM7YB, B018FGYJ0M, B07CKCWKY3, B07K69CXQ2, B07KHCNSN7, B07BRHMQXV,...","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,vibeWEAR,$9.29,B00BNFNU0S,"[https://images-na.ssl-images-amazon.com/images/I/41q7eG0ju1L._SR38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/41q7eG0ju1L.jpg],{},"B078YS9SKB,B07JMQM7YB,B01BYAZCXO,B01BYAJUYQ,B003EIK136,B005SHDGI8,B004UG4I3K,B005SHD0NO,B01BYATL..."
809383,"[Sports & Outdoors, Fan Shop, Jewelry & Watches, Sizes 6.5-16 - Men's and Women's Sized Rings - ...",,[Promise Wedding Frienship ring. Buy women's or men's sizes from this listing. This buck and doe...,,southern designs Buck and Doe Ring Band Ring in His and Her Sizes - Tungsten,"[B0147EN9U2, B014LPWOAS]",,Southern Designs,"[Sizes 6.5-16 - Men's and Women's Sized Rings - All Rings are 8mm width, High Quality Tungsten -...","1,304,632 in Clothing, Shoes & Jewelry (","[B014LPWOAS, B0147EN9U2, B07DKXG1HX, B00LML9FGQ, B07KYP9J65, B07HT32TFH, B01N5NGFI6, B011NZQWXY,...",Sports & Outdoors,,southern designs,$14.95,B0147E5NVA,[],[],{},"B0147EN9U2,B014LPWOAS"
809377,"[Sports & Outdoors, Fan Shop, Jewelry & Watches, Women's Sizes 4-12.5 Country or Southern Barbie...",,[His Doe Ring! Great for any country or southern girl! Women's Ring Only! Matching Men's Ring Av...,,His Doe Ring &quot;for Women&quot; - Tungsten - Couples Ring Set - Hunting - Ladies Ring,"[B0147E5NRE, B014LPWOAS, B07DKXG1HX]",,Southern Designs,"[Women's Sizes 4-12.5 Country or Southern Barbie Jewelry, High Quality Tungsten - His Doe Ring -...","1,503,788 in Clothing, Shoes & Jewelry (","[B014LPWOAS, B0147E5NRE, B07DKXG1HX, B07HSX68F7, B011NZQWXY, B01N2KJYPF, B07KT7XYXB, B0749MTN13,...",Sports & Outdoors,,southern designs,$18.95,B0147EN9SO,[],[],{},"B0147E5NRE,B014LPWOAS,B07DKXG1HX"
914911,"[Sports & Outdoors, Outdoor Recreation, Outdoor Clothing, Men, Accessories, Sport Watches, Class...",,"[<b>SONGDU Watches</b><br> A unique watch company that is infinitely driven, by the ever evolvin...",,SONGDU Date Black Multifunction Chronograph Mens Watches Stopwatch Rose Gold With Leather Strap ...,"[B0719D5GXB, B078N4DPT7, B0777HBD4D, B01F290W92, B07MJ1T5ZR, B071FTG25L, B07MDT2Q46, B07FPJDJ29,...",,songdu binge

<span style='background:orange;font-size:40px;'>Alternative Methodology to find Alsobuys exist?</span>

In [74]:

df_metadata["also_buy"].apply(lambda x: 1 if len(x) == 0 else 0).sum()

0

<span style='background:orange;font-size:40px;'>Build Subset df to eliminate asins with no alsobuys</span>

In [75]:
df_alsobuys = df_metadata[df_metadata.also_buy.str.len() > 0]

In [76]:
len(df_alsobuys)
print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

249165

There are 249,165 asins with at least one also_buy co-purchase 



<span style='background:orange;font-size:70px;'>At this point there is no reason to keep working with the full category metadata set, so we can eliminate any rows with ZERO also-buys</span>

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Turn this flag on to replace full metadata df with subset limited to asins with alsobuys</b></p>
</div>

In [77]:
#df_metadata = df_alsobuys
df_metadata['asin'].str.strip()
len(df_metadata)
len(df_metadata)==len(df_alsobuys)

956477    B01H3MRD48
420887    B00BNFNU0S
809383    B0147E5NVA
809377    B0147EN9SO
914911    B01DJ61XWK
             ...    
343203    B008CIHU2I
343205    B008CIO4FO
343211    B008CJ2946
343212    B008CJFH2M
962299    B01HJGAJ9O
Name: asin, Length: 249165, dtype: object

249165

True

In [78]:
df_metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys
956477,"[Sports & Outdoors, Outdoor Recreation, Outdoor Clothing, Men, Accessories, Sport Watches, Packa...",,"[Features: <br/>Shows time digital numbers <br/>Display time in numbers indicate the hour, minut...",,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,"[B07J9QSVVX, B01LNJDK5S, B017DC1X6C, B01D2W7NFW, B017CQLJ04, B01H1LCCG0, B01LA9B65W, B008031QKW]",,yunanwa,"[Package Included:10 Pack Led Watches 2016 Update Version, Additional RED LED function to show t...","680,789 in Clothing, Shoes & Jewelry (","[B01LNJDK5S, B06XD7FZWV, B01D2W7NFW, B07J9QSVVX, B01EN7RUHG, B01CJLLM6S, B017DC1X6C]","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,yunanwa,$19.99,B01H3MRD48,"[https://images-na.ssl-images-amazon.com/images/I/41Swby3LkzL._SR38,50_.jpg, https://images-na.s...","[https://images-na.ssl-images-amazon.com/images/I/41Swby3LkzL.jpg, https://images-na.ssl-images-...",{},"B07J9QSVVX,B01LNJDK5S,B017DC1X6C,B01D2W7NFW,B017CQLJ04,B01H1LCCG0,B01LA9B65W,B008031QKW"
420887,"[Sports & Outdoors, Fan Shop, Jewelry & Watches]",,"[As fun as it is functional, this slap watch features a stiff band that ""slaps"" around your wris...",,Silicone Slap-On Watch - Quartz Authentic Slap Watch - Blue,"[B078YS9SKB, B07JMQM7YB, B01BYAZCXO, B01BYAJUYQ, B003EIK136, B005SHDGI8, B004UG4I3K, B005SHD0NO,...",,vibeWEAR,"[100% Silicone Band - Fits 99% of Wrists, Quartz Time-Piece for 100% Accuracy, Lightweight and C...","529,043 in Clothing, Shoes & Jewelry (","[B078YS9SKB, B07K68NDPY, B07JMQM7YB, B018FGYJ0M, B07CKCWKY3, B07K69CXQ2, B07KHCNSN7, B07BRHMQXV,...","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,vibeWEAR,$9.29,B00BNFNU0S,"[https://images-na.ssl-images-amazon.com/images/I/41q7eG0ju1L._SR38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/41q7eG0ju1L.jpg],{},"B078YS9SKB,B07JMQM7YB,B01BYAZCXO,B01BYAJUYQ,B003EIK136,B005SHDGI8,B004UG4I3K,B005SHD0NO,B01BYATL..."
809383,"[Sports & Outdoors, Fan Shop, Jewelry & Watches, Sizes 6.5-16 - Men's and Women's Sized Rings - ...",,[Promise Wedding Frienship ring. Buy women's or men's sizes from this listing. This buck and doe...,,southern designs Buck and Doe Ring Band Ring in His and Her Sizes - Tungsten,"[B0147EN9U2, B014LPWOAS]",,Southern Designs,"[Sizes 6.5-16 - Men's and Women's Sized Rings - All Rings are 8mm width, High Quality Tungsten -...","1,304,632 in Clothing, Shoes & Jewelry (","[B014LPWOAS, B0147EN9U2, B07DKXG1HX, B00LML9FGQ, B07KYP9J65, B07HT32TFH, B01N5NGFI6, B011NZQWXY,...",Sports & Outdoors,,southern designs,$14.95,B0147E5NVA,[],[],{},"B0147EN9U2,B014LPWOAS"
809377,"[Sports & Outdoors, Fan Shop, Jewelry & Watches, Women's Sizes 4-12.5 Country or Southern Barbie...",,[His Doe Ring! Great for any country or southern girl! Women's Ring Only! Matching Men's Ring Av...,,His Doe Ring &quot;for Women&quot; - Tungsten - Couples Ring Set - Hunting - Ladies Ring,"[B0147E5NRE, B014LPWOAS, B07DKXG1HX]",,Southern Designs,"[Women's Sizes 4-12.5 Country or Southern Barbie Jewelry, High Quality Tungsten - His Doe Ring -...","1,503,788 in Clothing, Shoes & Jewelry (","[B014LPWOAS, B0147E5NRE, B07DKXG1HX, B07HSX68F7, B011NZQWXY, B01N2KJYPF, B07KT7XYXB, B0749MTN13,...",Sports & Outdoors,,southern designs,$18.95,B0147EN9SO,[],[],{},"B0147E5NRE,B014LPWOAS,B07DKXG1HX"
914911,"[Sports & Outdoors, Outdoor Recreation, Outdoor Clothing, Men, Accessories, Sport Watches, Class...",,"[<b>SONGDU Watches</b><br> A unique watch company that is infinitely driven, by the ever evolvin...",,SONGDU Date Black Multifunction Chronograph Mens Watches Stopwatch Rose Gold With Leather Strap ...,"[B0719D5GXB, B078N4DPT7, B0777HBD4D, B01F290W92, B07MJ1T5ZR, B071FTG25L, B07MDT2Q46, B07FPJDJ29,...",,songdu binge

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">There are multiple ways to build an edgelist</b></p>
</div>

# Create Dictionary Form of Co-Purchases by Asin

In [79]:
# There are two different dictionary structures


## Dictionary where the keys are ['asin','title', 'also_buy'] and the values are a dictionary with indexed, attrb

In [80]:
alsodict = df_metadata[['asin','title', 'also_buy']].to_dict()


In [81]:
# [{0: '0011300000',
#   1: '0043396828',
#   2: '0060009810',

#list(alsodict.values())[:1]

# [{0: 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#   1: 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#   2: 'One Hot Summer',
#   3: 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',
#   4: 'sex.lies.murder.fame.: A Novel',
#   5: 'College Physics',

In [82]:
## Dict where the ASIN is the key and 

In [83]:
#metadf[['asin','title', 'also_buy']].dropindex()
meta_df = df_metadata[['asin','title', 'also_buy']].set_index('asin')
#meta_dffinal = df_graph[['asin','title', 'also_buy']].set_index('asin')
meta_dict = meta_df.to_dict()
#meta_dictfinal = meta_dffinal.to_dict()


## Dict where the keys are ['title', 'also_buy'] and the values are [asin, attr]

In [84]:
meta_df.head()
#list(meta_dict.items())[0:1]
meta_dict.keys()

,title,also_buy
asin,,
B01H3MRD48,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,"[B07J9QSVVX, B01LNJDK5S, B017DC1X6C, B01D2W7NFW, B017CQLJ04, B01H1LCCG0, B01LA9B65W, B008031QKW]"
B00BNFNU0S,Silicone Slap-On Watch - Quartz Authentic Slap Watch - Blue,"[B078YS9SKB, B07JMQM7YB, B01BYAZCXO, B01BYAJUYQ, B003EIK136, B005SHDGI8, B004UG4I3K, B005SHD0NO,..."
B0147E5NVA,southern designs Buck and Doe Ring Band Ring in His and Her Sizes - Tungsten,"[B0147EN9U2, B014LPWOAS]"
B0147EN9SO,His Doe Ring &quot;for Women&quot; - Tungsten - Couples Ring Set - Hunting - Ladies Ring,"[B0147E5NRE, B014LPWOAS, B07DKXG1HX]"
B01DJ61XWK,SONGDU Date Black Multifunction Chronograph Mens Watches Stopwatch Rose Gold With Leather Strap ...,"[B0719D5GXB, B078N4DPT7, B0777HBD4D, B01F290W92, B07MJ1T5ZR, B071FTG25L, B07MDT2Q46, B07FPJDJ29,..."


dict_keys(['title', 'also_buy'])

In [85]:
# [('title',
#   {'0011300000': 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#    '0043396828': 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#    '0060009810': 'One Hot Summer',
#    '0060219602': 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',

In [86]:
alsodict.keys()
meta_dict.keys()
#meta_dictfinal.keys()
type(meta_dict['also_buy'])
#len(meta_dictfinal['also_view'])
#meta_df
#meta_dict['also_buy']

dict_keys(['asin', 'title', 'also_buy'])

dict_keys(['title', 'also_buy'])

dict

# ASINS Sorted/Ranked by Number of Also_Buy

In [87]:
def sort_by_values_len(dict):
    dict_len= {key: len(value) for key, value in dict.items()}
    import operator
    sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
    sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
    return sorted_dict

# Sorted Dictionary where key is asin and value is alsobuys list

In [88]:
alsobuys_rnk = sort_by_values_len(meta_dict['also_buy'])

In [89]:
#sorted(meta_dict['also_buy'].items(), key=lambda e: len(e))
type(alsobuys_rnk)
len(alsobuys_rnk)

type(alsobuys_rnk[0])
len(alsobuys_rnk[0:2])

list

249165

dict

2

In [90]:

# This Dictionary is 2996 rows less which is correct because it has removed the duplicate rows and combined the values

## Verification that the dictionary is ranked and the 100,776th row has no alsobuys (therefore doesn't exist)

In [91]:
# You can see how the dictionary is ranked by the number of alsobuys
alsobuys_rnk[1].values()
alsobuys_rnk[len(alsobuys_rnk)-1].values()
alsobuys_rnk[len(alsobuys_rnk)].values()


dict_values([['B001445F42', 'B00LXCLE72', 'B000OMJWNA', 'B005UGEB6S', 'B000KOTTKS', 'B001CAW2DK', 'B000C6MI6Q', 'B002IV6RCO', 'B000KOPV5U', 'B001GJ35VU', 'B004XAH7TK', 'B0036PUHN6', 'B0012RJTUW', 'B0000BYASB', 'B00144784C', 'B00BBLM8ZW', 'B001KH205A', 'B0000AXNJO', 'B006GWVBQ8', 'B000N9T7JI', 'B000XBFBZ0', 'B000XBCURW', 'B00166NCGG', 'B002GUELES', 'B000E8FSTQ', 'B00CXADOZ8', 'B001HWPZ6Y', 'B0030HMYFE', 'B000N9O6N0', 'B003DK2LM4', 'B0000AXVG9', 'B001445EUW', 'B007TMXTT6', 'B009CEMPJY', 'B008N3JA2U', 'B000N9Q0PC', 'B0009H1AMG', 'B015VSN2SW', 'B01M31ENPF', 'B00230HTKE', 'B000XBHAV8', 'B000BQUG9I', 'B0007TQWJM', 'B0000AXZ2I', 'B000UJG0LE', 'B01N0IXH48', 'B0000ATYSE', 'B0000AXYA0', 'B005C5C3GC', 'B00S4HFGUA', 'B06Y63WT9K', 'B07B8TNYV3', 'B00JGZGTDC', 'B000KOTYQC', 'B004UOUS1I', 'B00030MAL4', 'B0000AY2HN', 'B000MT95R8', 'B004B7UMOM', 'B008CE0TNE', 'B076PBKY61', 'B07CSYP3LH', 'B0000AY4YJ', 'B0000AYAUF', 'B003DK3GSC', 'B006GJYLFY', 'B000N9ZJ7M', 'B00CO5PSOC', 'B000OF91EW', 'B003CURWLA', 'B000J

dict_values([['B00GPE7GTS']])

IndexError: list index out of range

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge Set Manually</b></p>
</div>

In [92]:

edgeset = set()
for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]: 
        for alsobuy in alsobuys_rnk[i][asin]: # For Every Alsobuy in an alsobuy list
            edgeset.add(tuple(sorted((asin,alsobuy)))) # changes order of tuples 
        


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge LIST</b></p>
</div>

In [93]:
edgelist = []

for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]:
        for alsobuy in alsobuys_rnk[i][asin]:
            edgelist.append((asin, alsobuy))


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Note that they are of Different Lengths (probably list has duplicates)</b></p>
</div>

In [94]:
type(edgeset)
len(edgeset)
type(edgelist)
len(edgelist)
# The edgelist has 61,000 more than the set

set

6170606

list

6414520

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">When Creating a Set from the List this way, the number of edges is the same</b></p>
</div>


In [95]:
res = set(map(lambda x: tuple(sorted(x)), edgelist))

In [96]:
len(res) #Interesting same length as set above
res==edgeset

6170606

True

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">The above is proof that creating the set of edges is accurate and the list is not</b></p>
</div>


In [97]:
# Same number of asins
len(meta_dict['also_buy'].keys())
len(alsobuys_rnk)

249165

249165

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Annabels formula was built using the raw metadata file, not the df, so to make is less expensive I am lobbing off all rows we already know have no alsobuys<br>
<br>I already turned on the flag above to make this conversion so I am just reverting the df back to a list
<br>Create SUBSET METADATA TO LOWER COMPUTATION COST OF ANNABELS FORMULA</b></p>
</div>

In [99]:
metadatab = df_metadata.to_dict('records')

In [100]:
type(metadatab)
len(metadatab)

list

249165



<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Correct Edgelist Approach w Weights</b></p>
</div>


In [101]:
dict2 = {}

for item1 in metadatab: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict2: 
            dict2[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict2: 
            dict2[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict2:
            dict2[(item2,item1['asin'])] += 1
    list_of_pairs = [(item1['also_buy'][p1], item1['also_buy'][p2]) for p1 in range(len(item1['also_buy'])) for p2 in range(p1+1,len(item1['also_buy']))]
    for (item3,item4) in list_of_pairs: 
#          if(item3, item4) and (item4, item3) not in dict1: 
#              dict1[(item3,item4)] = 1
        if (item3,item4) in dict2: 
            dict2[(item3,item4)] += 1
            break
        elif (item4,item3) in dict2: 
            dict2[(item4,item3)] += 1

In [102]:
# In this formula we take all the pairs of items in a single also bought list and then check to see if they are already in the edgelist. If they are we increase their weight by 1
# This is correct because we are not adding any new edges we are just giving freqently seen alsobought added focus/weight

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Basic Unweighted Edgeweight Approach, identical to networkx and my approach above</b></p>
</div>


In [103]:
#len(dict1)
len(dict2)


#max(dict1.values())
max(dict2.values())





#dict(itertools.islice(dict2.items(), 10))
#sorted(dict2.items(), key=lambda x: x[1],reverse=True)

# 29319645
# 1818275
# 1818275
# 271
# 109
# 3

6170606

212


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px"> Create Edge set using the networkx built-in function</b></p>
</div>

In [104]:
import networkx as nx
g = nx.from_dict_of_lists(meta_dict['also_buy'])


In [105]:
len(meta_dict['also_buy'])

249165

# It has the same number of edges as our manual edgeliset but different number of nodes?

In [106]:
g.size()
type(g)
len(g)
len(g.nodes())
len(g.edges())
# This matches the manual method above

6170606

networkx.classes.graph.Graph

977973

977973

6170606

<span style='background:purple;font-size:30px;color:white'>Proof that the THREE different edgelist formulas result in the exact same edges</span>


In [107]:
len(pd.DataFrame(dict2.keys()))
len(pd.DataFrame(edgeset))

#dfedges = pd.DataFrame(list(dict2.keys()),columns=['source','target'])
dict2edges = pd.Series(dict2).rename_axis(['source', 'target']).reset_index(name='weight')
dfedgeset = pd.DataFrame(edgeset, columns=['source','target'] )


6170606

6170606

In [108]:
sourcedtwo = dict2edges['source'].unique()
targetdtwo = dict2edges['target'].unique()
dtwonodes = np.unique(np.concatenate([sourcedtwo, targetdtwo]))
len(dtwonodes)
dtwolist = sorted(list(dtwonodes))

977973

In [109]:
sourcees = dfedgeset['source'].unique()
targetes = dfedgeset['target'].unique()
esnodes = np.unique(np.concatenate([sourcees, targetes]))
len(esnodes)
eslist = sorted(list(esnodes))

977973

In [110]:
gnodes = sorted(list(g.nodes))

In [111]:
dtwolist==eslist==gnodes

True

***

In [112]:
type(edgeset)
set(itertools.islice(edgeset, 10))

set

{('1607432382', 'B005RGTBHU'),
 ('B004HIHQLM', 'B01A5IF6EQ'),
 ('B0052085K0', 'B0761JZJGY'),
 ('B006KU6VGQ', 'B07CSBPN9F'),
 ('B0084MBRWG', 'B01M3UTECQ'),
 ('B00D7OM0SG', 'B016N7Y7S4'),
 ('B00MPE9O06', 'B00MYL9X4M'),
 ('B014RG5TCQ', 'B01G59YNEI'),
 ('B017ILUIYG', 'B077QJNVKK'),
 ('B01BW7D1J6', 'B01MUEUWSS')}

<span style='background:purple;font-size:30px;color:white'>Build Final Edgelist DataFrame</span>


In [113]:
dfedges = dict2edges
len(dfedges)
dfedges.head()

#df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])

6170606

,source,target,weight
0,B01H3MRD48,B07J9QSVVX,1
1,B01H3MRD48,B01LNJDK5S,1
2,B01H3MRD48,B017DC1X6C,1
3,B01H3MRD48,B01D2W7NFW,1
4,B01H3MRD48,B017CQLJ04,1


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create a Lookup Key that concatenates source+target asin strings</b></p>
</div>


In [114]:
dfedges['lookup'] =  dfedges['source'] + dfedges['target']
dfedges.sort_values(by='weight', ascending=False)

,source,target,weight,lookup
83605,B0051LQ0X4,B00159O3NU,212,B0051LQ0X4B00159O3NU
619459,B00NG1ECBS,B00D3PNZB0,211,B00NG1ECBSB00D3PNZB0
39317,B000FE9C14,B000OF5LWS,196,B000FE9C14B000OF5LWS
79340,B0048970A2,B001BR4RO6,191,B0048970A2B001BR4RO6
790775,B00NW23YBA,B0050LUBZ8,168,B00NW23YBAB0050LUBZ8
...,...,...,...,...
2211521,B01H1SK1SO,B01IQ85QYM,1,B01H1SK1SOB01IQ85QYM
2211519,B01H1SK1SO,B077GN7KDW,1,B01H1SK1SOB077GN7KDW
2211514,B01H1SK1SO,B075C9QXQR,1,B01H1SK1SOB075C9QXQR
2211510,B01H1SK1SO,B072K3F6QZ,1,B01H1SK1SOB072K3F6QZ


In [115]:

#df_weights = df_weights.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [116]:
## These needs to be an innerjoin followed by a left join with drop na, because if either source or node is missing metadata we need to drop

In [117]:
dfmerge1 = dfedges.merge(df_masterMETA.rename(columns={'title': 'target_title'}), how='inner', left_on=['target'], right_on=['asin'])


In [118]:
len(dfmerge1)
dfmerge1.head()


1861152

,source,target,weight,lookup,target_title,asin
0,B01H3MRD48,B017DC1X6C,1,B01H3MRD48B017DC1X6C,KitMax (TM) Wholesale Pack of 12 Pcs Assorted Color Sport LED Digital Silicone Jelly Band Bracel...,B017DC1X6C
1,B01H3MRD48,B01D2W7NFW,1,B01H3MRD48B01D2W7NFW,CdyBox 8 Pack Wholesale Men Women Kids Digital Wristwatch Touch Screen LED Bracelet Silicone Ban...,B01D2W7NFW
2,B01H3MRD48,B017CQLJ04,1,B01H3MRD48B017CQLJ04,KitMax (TM) Wholesale Pack of 6 Pcs Assorted Colors Unisex Mirror Automatic LED Digital Sport Wr...,B017CQLJ04
3,B01H3MRD48,B01H1LCCG0,1,B01H3MRD48B01H1LCCG0,KIOP Men's Luxury Fashion Faux Leather Mens Blue Ray Glass Quartz Analog Watches,B01H1LCCG0
4,B01H3MRD48,B008031QKW,1,B01H3MRD48B008031QKW,NYKKOLA Wholesale 12 Assorted Geneva Women's Watches,B008031QKW


In [119]:
df_EDGES = dfmerge1.merge(df_masterMETA.rename(columns={'title': 'src_title'}), how='left', left_on=['source'], right_on=['asin']).dropna()[['source', 'target','weight', 'src_title', 'target_title','lookup']].copy()

In [120]:
len(df_EDGES)
df_EDGES.head()


1880312

,source,target,weight,src_title,target_title,lookup
0,B01H3MRD48,B017DC1X6C,1,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,KitMax (TM) Wholesale Pack of 12 Pcs Assorted Color Sport LED Digital Silicone Jelly Band Bracel...,B01H3MRD48B017DC1X6C
1,B01H3MRD48,B01D2W7NFW,1,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,CdyBox 8 Pack Wholesale Men Women Kids Digital Wristwatch Touch Screen LED Bracelet Silicone Ban...,B01H3MRD48B01D2W7NFW
2,B01H3MRD48,B017CQLJ04,1,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,KitMax (TM) Wholesale Pack of 6 Pcs Assorted Colors Unisex Mirror Automatic LED Digital Sport Wr...,B01H3MRD48B017CQLJ04
3,B01H3MRD48,B01H1LCCG0,1,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,KIOP Men's Luxury Fashion Faux Leather Mens Blue Ray Glass Quartz Analog Watches,B01H3MRD48B01H1LCCG0
4,B01H3MRD48,B008031QKW,1,Yunanwa 10 Pack Wholesale Women Men Kids LED Sports Bracelet Wrist Watch Silicone Band Digital U...,NYKKOLA Wholesale 12 Assorted Geneva Women's Watches,B01H3MRD48B008031QKW


In [121]:
df_EDGES[df_EDGES.weight.isnull()]

,source,target,weight,src_title,target_title,lookup


In [122]:
df_EDGES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880312 entries, 0 to 1880920
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   source        object
 1   target        object
 2   weight        int64 
 3   src_title     string
 4   target_title  string
 5   lookup        object
dtypes: int64(1), object(3), string(2)
memory usage: 100.4+ MB


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Check for duplicates via lookup</b></p>
</div>


In [123]:
df_EDGES[df_EDGES.duplicated(subset=['lookup', 'weight'], keep=False)].sort_values(by='lookup', ascending=False).reset_index(drop=True)

,source,target,weight,src_title,target_title,lookup
0,B01HJ5QPQ6,B0002XN6CO,1,Under Armour Boys' Gameday Armour Forearm Shiver,Oakley Football Shield,B01HJ5QPQ6B0002XN6CO
1,B01HJ5QPQ6,B0002XN6CO,1,Under Armour Boys' Gameday Armour Forearm Shiver,Oakley Football Shield,B01HJ5QPQ6B0002XN6CO
2,B01HJ4DN08,B00019N0PW,1,Savage Gear 3D Topwater Duck,"Mepps BM HO-BO MuskyKiller-Bucktail, 3/4-Ounce, Hot Orange",B01HJ4DN08B00019N0PW
3,B01HJ4DN08,B00019N0PW,1,Savage Gear 3D Topwater Duck,"Mepps BM HO-BO MuskyKiller-Bucktail, 3/4-Ounce, Hot Orange",B01HJ4DN08B00019N0PW
4,B01HJ2MI56,B0001M4C06,1,Southland Archery Supply SAS Premium Neoprene Bow Sling,G5 Outdoors G-Lock Blu Glue 20g #00983,B01HJ2MI56B0001M4C06
...,...,...,...,...,...,...
91057,1938393090,B0000E5LTR,1,"Shuteye Ridge, California",Black Diamond White Gold Loose Chalk,1938393090B0000E5LTR
91058,1926644425,B00004Z5KG,1,Only1MILLION Waist Trimmer Belt and Smartphone Neoprene Sleeve - Waist Sauna for Accelerated Wei...,Belkin RG58 50-Ohm Thin Ethernet Coaxial Cable with BNC to BNC Male Connectors (6 Feet),1926644425B00004Z5KG
91059,1926644425,B00004Z5KG,1,Only1MILLION Waist Trimmer Belt and Smartphone Neoprene Sleeve - Waist Sauna for Accelerated Wei...,Belkin RG58 50-Ohm Thin Ethernet Coaxial Cable with BNC to BNC Male Connectors (6 Feet),1926644425B00004Z5KG
91060,0000031895,B00009IMCT,1,Girls Ballet Tutu Neon Blue,Manhattan Toy Dr. Seuss Cat in the Hat 12&quot; Soft Plush Toy,0000031895B00009IMCT


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Eliminate Duplicate Edges</b></p>
</div>


In [124]:
df_EDGES = df_EDGES.drop_duplicates(subset=['lookup', 'weight'], keep='last')

In [125]:
len(df_EDGES)


1833697

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Eliminate Duplicate rows where the titles for source and target are identical and weights (but different asins)</b></p>
</div>


In [126]:
df_EDGES = df_EDGES.drop_duplicates(subset=['src_title', 'target_title', 'weight'], keep='last')

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Drop Items with same title on both source/target but different weights and keep the largest weight.  (should figure out way to sum)</b></p>
</div>


In [127]:
df_EDGES = df_EDGES.sort_values(by=['src_title', 'target_title','weight'], ascending=False).drop_duplicates(subset=['src_title', 'target_title'], keep='first')

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Export Final Cleaned up Edgeslist to CSV</b></p>
</div>


In [131]:
# compression_opts = dict(method='zip',
#                         archive_name='FINALELECTRONICS.csv')  
df_EDGES.to_csv('NOV24_Sports.csv', index=False)  

In [ ]:
#df_EDGES[df_EDGES.duplicated(subset=['src_title', 'target_title'], keep=False)].sort_values(by=['src_title', 'target_title','weight'], ascending=False)#.reset_index(drop=True)

#df_EDGES.sort_values(by=['src_title', 'target_title','weight'], ascending=False)[df_EDGES.duplicated(subset=['src_title', 'target_title'], keep=False)]#.reset_index(drop=True)

In [128]:
df_EDGES[df_EDGES.duplicated(subset=['src_title', 'target_title'], keep=False)].sort_values(by=['src_title', 'target_title','weight'], ascending=False)#.reset_index(drop=True)

,source,target,weight,src_title,target_title,lookup


In [129]:
#df_EDGES.sort_values('source')
df_EDGES[df_EDGES['target']=='B0002E1UEO']

,source,target,weight,src_title,target_title,lookup


In [130]:
#pd.set_option('display.max_rows', 100)
#df_EDGES[df_EDGES.duplicated(subset=['src_title', 'source','target', 'target_title'])]
df_EDGES[df_EDGES.duplicated(subset=['src_title', 'target_title'], keep=False)].sort_values('weight', ascending=False).head(100)
#.groupby(['src_title', 'target_title'])  #

#df_EDGES.groupby(['src_title', 'target_title']).size()  

,source,target,weight,src_title,target_title,lookup
